In [ ]:
# vectorDB에 이미지 자료를 I/O
# 이미지 벡터화 처리는 Resnet 모델 사용
# 이미지 -> 벡터 DB -> 저장 -> 설치

# 이미지 벡터화(Image Embedding)와 이미지 기반 유사도 검색을 구현하는 실무적인 예시. 텍스트 RAG에서 한 단계 더 나아가,
# 컴퓨터 비전(Computer Vision) 모델을 벡터 DB와 연동하는 멀티모달(Multimodal) 시스템
!pip install chromadb sentence-transformers torchvision torch pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.4 MB/s eta

In [ ]:
import os
import torch
from torchvision import transforms # 이미지 전처리(Pre-processing)를 위한 모듈
from PIL import Image # 이미지 파일 처리를 위한 라이브러리
from chromadb import PersistentClient # 벡터 DB 영구 저장을 위한 클라이언트
from torchvision.models import resnet18 # ResNet18 모델을 임포트

# torchvision.models에 포함된 모든 ResNet 계열 모델 이름 출력 (확인용)
import torchvision.models as models
all_models = dir(models)
resnet_models = [name for name in all_models if 'resnet' in name.lower()]
print(resnet_models)

['ResNet', 'ResNet101_Weights', 'ResNet152_Weights', 'ResNet18_Weights', 'ResNet34_Weights', 'ResNet50_Weights', 'Wide_ResNet101_2_Weights', 'Wide_ResNet50_2_Weights', 'resnet', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'wide_resnet101_2', 'wide_resnet50_2']


In [ ]:
# 1. 이미지를 벡터화(Embedding)할 모델 설정: ResNet18 사용
model = resnet18(weights='ResNet18_Weights.DEFAULT')    # ImageNet 데이터셋으로 사전 학습된 가중치 로드
model.eval()    # 모델을 평가 모드로 전환. 학습(train)이 아닌 추론(inference)용으로 설정
model = torch.nn.Sequential(*list(model.children())[:-1])   # 맨 마지막 FC층은 제거(분류모델이 아니라 임베딩만 원함)
print(model)

# FC를 제외한 나머지 레이어들을 순차적으로 묶은 새로운 모델로 재구성
# 이미지를 불러 ResNet18을 이용해 512차원 벡터로 반환하는 함수

# 2. 이미지를 불러 ResNet18을 이용해 512차원 벡터로 반환하는 함수 정의
def image_to_vectorFunc(img_path):  # 2-1. 이미지 로딩 및 전처리
    image = Image.open(img_path).convert('RGB') # 이미지를 열고 RGB 3채널로 변환
    transform = transforms.Compose([
        transforms.Resize((224,224)),   # ResNet18 입력 이미지 크기(3, 224, 224)를 원함
        transforms.ToTensor()   # PIL.image를 PyTorch의 Tensor 형태로 변환 (0~255 값을 0.0~1.0으로 정규화)
    ])

    # 2-2. 텐서 형태 변환: (채널, 높이, 너비) -> (배치, 채널, 높이, 너비)
    tensor = transform(image).unsqueeze(0) #모델에 넣기 위해 배치차원 (앞에 1) 추가 (3,224,224)

    # 2-3. 모델 추론
    with torch.no_grad():   # 역전파(Backpropagation) 계산을 비활성화하여 메모리 및 속도 최적화
        # vec의 초기 shape: (1, 512, 1, 1) -> Conv Layer와 Global Average Pooling의 결과
                vec = model(tensor).squeeze().numpy()   #모든 크기가 1인 차원 제거 (예: (1, 512, 1, 1) -> (512,))
    print(f'{img_path} -> 벡터(앞 10개): {vec[:10]}')
    return vec  # 최종 512차원 특징 벡터 반환


# 3. 데이터 준비 및 ChromaDB 설정
# 이미지 경로 리스트 (경로 'pic' 폴더는 사용자 환경에 존재해야 함)
filenames = ['apple.jpeg', 'peach.jpeg', 'grape.jpeg']
image_files = [os.path.join('pic', name) for name in filenames]
ids = [f'img{i}' for i in range(len(image_files))]
print('ids :',ids)

# 벡터에 저장   
client = PersistentClient('./image_chroma') # 이미지 저장을 위한 새로운 DB 경로 설정
collection = client.get_or_create_collection(name='images')

# 4. 이미지 벡터화 및 DB 저장 (Indexing)
for img_id, img_path in zip(ids, image_files):
    if not os.path.exists(img_path):
        print(f'파일 없음: {img_path}')
        continue
    vec = image_to_vectorFunc(img_path) # 정의한 함수를 사용하여 512차원 벡터 생성
    collection.add(
        embeddings=[vec],   # 생성된 벡터
        documents=[img_path],   # 원본 문서(여기서는 파일 경로)
        ids=[img_id],
        metadatas=[{'filename':img_path}]   # 파일명을 메타데이터로 저장
    )

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [ ]:
# 5. 저장된 벡터 일부 출력 (검증)
record = collection.get(ids = ['img0'], include=['embeddings','documents','metadatas'])
print('documents : ', record['documents'][0])
print('metadatas : ', record['metadatas'][0])
print('embeddings 10개 : ', record['embeddings'][0][:10])

# 6. 이미지 유사도 검색 (Retrieval)
query_image_path = 'pic/apple.jpeg' # 검색에 사용할 질문(이미지) 지정
if not os.path.exists(query_image_path):
    print(f'검색 이미지가 없어요 : {query_image_path}')
else:
    query_vec = image_to_vectorFunc(query_image_path)   # 질문 이미지도 512차원 벡터로 변환
    results = collection.query(
        query_embeddings=[query_vec],   # 질문 벡터로 검색
        n_results=3, # 가장 유사한 이미지 3개 반환 요청
        include=['metadatas', 'distances', 'documents']
    )
# 7. 검색 결과 출력
print(f'검색 이미지 : {query_image_path}')
print('유사 이미지 :')
# 결과 분석: 'apple.jpeg'가 가장 작은 거리(가장 유사)로 반환되고, 'peach.jpeg' 또는 'grape.jpeg'가 그 다음으로 반환됨
for doc, meta, dist in zip(results['documents'][0], results['metadatas'][0],results['distances'][0]):
    print(f'-파일명 :{meta['filename']} (유사도 거리 : {dist:.4f})')

documents :  pic/apple.jpeg
metadatas :  {'filename': 'pic/apple.jpeg'}
embeddings 10개 :  [2.74511003 1.38714886 0.48933688 1.52329242 0.20173761 1.49299932
 0.51078451 0.9898755  0.7163552  0.24798128]
pic/apple.jpeg -> 벡터(앞 10개): [2.74511    1.3871489  0.48933688 1.5232924  0.20173761 1.4929993
 0.5107845  0.9898755  0.7163552  0.24798128]
검색 이미지 : pic/apple.jpeg
유사 이미지 :
-파일명 :pic/apple.jpeg (유사도 거리 : 0.0000)
-파일명 :pic/peach.jpeg (유사도 거리 : 394.2893)
-파일명 :pic/grape.jpeg (유사도 거리 : 684.0030)


추가 개념 정리: 이미지 벡터 I/O의 핵심

1. 이미지 임베딩: 특징 추출기 (Feature Extractor)
텍스트 임베딩 모델(SentenceTransformer)이 문장의 의미를 추출하듯, CNN 모델(ResNet18)은 이미지의 시각적 특징을 추출하여 벡터로 변환.

전이 학습 (Transfer Learning)의 활용: ResNet18은 이미 수백만 장의 이미지(ImageNet)를 통해 사물의 형태, 색상, 질감 같은 시각적 특징을 학습한 모델.


분류층 제거: 원래 ResNet18은 이미지를 1000개의 카테고리로 분류하는 모델. 마지막 Fully Connected (FC) 층을 제거하면, 분류 결과 대신 그 직전 단계에서 추출된 고수준의 특징 벡터(Feature Vector), 즉 512차원 임베딩을 얻게 됩니다. 이 임베딩은 이미지의 시각적 '의미'를 담고 있습니다.

2. 이미지 전처리 파이프라인 (Transforms)
딥러닝 모델에 이미지를 입력하기 전에는 반드시 모델이 요구하는 형태로 변환.

transforms.Resize((224, 224)): ResNet과 같은 CNN 모델들은 학습 시 정해진 크기(대부분 224x224 픽셀)의 이미지만 입력으로 받도록 설계. 입력 전에 이 크기로 맞춤.

transforms.ToTensor(): PIL 이미지를 PyTorch 텐서(Tensor)로 변환하는 과정에서 픽셀 값(0~255)을 0.0~1.0 사이의 실수로 정규화(Normalization). 이는 딥러닝 학습 및 추론의 안정성을 위해 필수적.

unsqueeze(0): (채널, 높이, 너비) 형태의 텐서 앞에 배치(Batch) 차원(1)을 추가하여 모델이 요구하는 (1, 채널, 높이, 너비) 형태(2차원 배열과 유사한 개념)로 만듦.

3. 이미지 유사도 검색의 의미
텍스트 검색이 "파이썬이 뭐냐"라는 질문 벡터와 "파이썬은 언어다"라는 문서 벡터의 유사도를 측정했다면, 이미지 검색은 "질문 사과 이미지" 벡터와 "저장된 복숭아 이미지" 벡터의 유사도를 측정.

유사도 판단: 두 이미지의 임베딩 벡터가 가까울수록 시각적으로 유사한 특징(예: 형태, 색상, 종류)을 가진 것으로 판단하여 높은 순위로 반환.

멀티모달 확장: 이 기술을 확장하면 "사과 그림"을 벡터화하여 사과에 대한 설명이 담긴 텍스트 벡터와 비교하는 크로스-모달(Cross-Modal) 검색도 가능.